In [22]:
from azureml.core import Workspace, Experiment

print("Accessing the workspace form job...")
ws = Workspace.from_config("./config")

print("Accessing the Adult Income dataset...")
input_ds = ws.datasets.get("IncomeTrunc")

Accessing the workspace form job...
Accessing the Adult Income dataset...


In [23]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name="Myenviron")
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', 'pip', 'pandas'],
                                     pip_packages=['azureml-defaults', 'azureml-interpret'])

myenv.python.conda_dependencies = myenv_dep
print("Registering the environment...")
myenv.register(ws)

Registering the environment...


{
    "assetId": "azureml://locations/westus/workspaces/54b058d1-e724-4986-aece-5eb3256cb2e6/environments/Myenviron/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240304.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "Myenviron",
    "python": {


In [24]:
cluster_name = "my-cluster-001"

from azureml.core.compute import AmlCompute
print("Accessing the compute cluster...")

if cluster_name not in ws.compute_targets:
    print("Creating the compute cluster with name: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(
                                     vm_size="STANDARD_D11_V2",
                                     max_nodes=2)

    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster = ws.compute_targets[cluster_name]
    print(cluster_name, ", compute cluster found. Using it...")

Accessing the compute cluster...
my-cluster-001 , compute cluster found. Using it...


In [25]:
from azureml.core import ScriptRunConfig
print("Creating the ScriptRunConfig....")
script_config = ScriptRunConfig(source_directory=".",
                                script="360 - Model explain script.py",
                                arguments = ['--input-data', input_ds.as_named_input('raw_data')],
                                environment=myenv,
                                compute_target=cluster)

Creating the ScriptRunConfig....


In [26]:
print("Creating the experiment")
new_experiment = Experiment(workspace=ws, name="Explainer_Exp001")

print("Submitting the experiment...")
new_run = new_experiment.submit(config=script_config)

new_run.wait_for_completion(show_output=True)

Creating the experiment
Submitting the experiment...
RunId: Explainer_Exp001_1712521424_c648b846
Web View: https://ml.azure.com/runs/Explainer_Exp001_1712521424_c648b846?wsid=/subscriptions/49e690c2-0fc4-42a6-ba64-9ea0fa04ddda/resourcegroups/AzureMLsdkgroup/workspaces/slavasdkworkspace&tid=77487836-da45-4554-ad65-1775dca67ca5

Streaming user_logs/std_log.txt

Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.06254386901855469 seconds
Traceback (most recent call last):
  File "360 - Model explain script.py", line 13, in <module>
    df = new_run.input_datasets['raw_data'].to_pandas_dataframe()
AttributeError: 'FileDataset' object has no attribute 'to_pandas_dataframe'


Execution Summary
RunId: Explainer_Exp001_1712521424_c648b846
Web View: https://ml.azure.com/runs/Explainer_Exp001_1712521424_c648b846?wsid=/subscriptions/49e690c2-0fc4-42a6-ba64-9ea0fa04ddda/resourcegroups/AzureMLsdkgroup/workspaces/slavasdkworkspace&tid=77487836-da4

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_119283618be4fc1236091a5cca648be0/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"360 - Model explain script.py\", line 13, in <module>\n    df = new_run.input_datasets['raw_data'].to_pandas_dataframe()\nAttributeError: 'FileDataset' object has no attribute 'to_pandas_dataframe'\n\n",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z",
    "componentName": "CommonRuntime"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution failed. User process '/azureml-envs/azureml_119283618be4fc1236091a5cca648be0/bin/python' 